In [1]:
from pathlib import Path
import urllib
import timm
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import torch
import sys
import os

/Users/aaronestes/anaconda3/envs/hello_world_poetry/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### TODO: Eventually we want to get the image from user's camera

In [3]:
model_type = 'inception_v3'
## if we set is_pretrained to False,
## model will tell us there's a 100% chance Alex's german shepard is a wallaby
is_pretrained = True

def create_model():
    return timm.create_model(model_type, pretrained=is_pretrained)

model = create_model()

In [4]:
def eval_model():
    return model.eval()

eval_model()

InceptionV3(
  (Conv2d_1a_3x3): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (Conv2d_2a_3x3): ConvNormAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (Conv2d_2b_3x3): ConvNormAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (Pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): ConvNormAct(
    (conv): Conv2d(64, 80, kernel_size

In [5]:
default_config = {}

def create_config():
    try:
        return resolve_data_config(default_config, model=model)
    except AssertionError:
        print('First, we need to create our pyTorch IMage Model (TIMM)!')
        sys.exit()
        
config = create_config()

In [6]:
transform = create_transform(**config)

In [7]:
base_folder = 'test_images'
cwd = Path.cwd()
fn = 'alex_dog_german_shepherd.jpg'
filename = Path.home().joinpath(cwd, base_folder, fn)

def find_file(f):
    if os.path.isfile(f):
        return True
    return False

file_found = find_file(filename)

In [8]:
def open_and_convert_img_rgb():
    if file_found:
        return Image.open(filename).convert('RGB')
    else:
        print(f'FILE: {filename} NOT FOUND!')
        sys.exit()

img = open_and_convert_img_rgb()

In [9]:
def create_tensor():
    return transform(img).unsqueeze(0)

tensor = create_tensor()

In [10]:
def get_output():
    with torch.no_grad():
        return model(tensor)

output = get_output()

In [11]:
def compute_probabilities():
    return torch.nn.functional.softmax(output[0], dim=0)

probabilities = compute_probabilities()

In [12]:
imagenet_classes_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_classes_filename = "imagenet_classes.txt"

def get_imagenet_classes():
    ## imports a list of all the different classes (1000 in total) the CNN is trained upon
    urllib.request.urlretrieve(imagenet_classes_url, imagenet_classes_filename)

# get_imagenet_classes()

In [13]:
def structure_imagenet_classes():
    with open(imagenet_classes_filename, "r") as f:
        return [s.strip() for s in f.readlines()]    

categories = structure_imagenet_classes()

In [14]:
n = 3

def get_most_probable():
    return torch.topk(probabilities, n)

topn_prob, topn_catid = get_most_probable()

In [15]:
def display_results():
    for i in range(topn_prob.size(0)):
        category_name = categories[topn_catid[i]]
        prob = str(round(float(topn_prob[i].item()*100), 1)) + "%"
        print(category_name, prob)

display_results()

German shepherd 71.0%
kelpie 8.0%
malinois 6.8%
